In [1]:
import numpy as np
import scipy.optimize as scyopt
import matplotlib.pyplot as plt
import cProfile

import pyaccel, sirius
import mathphys as mp
%load_ext autoreload
%autoreload 1
%aimport lattice_errors

In [ ]:
acc      = sirius.SI_V10.create_accelerator()
fam_data = sirius.SI_V10.get_family_data(acc)

In [ ]:
um, mrad, percent = 1e-6, 1e-3, 1e-2
config = dict({'mags':dict(),'girder':dict()})

config['mags']['quads'] = dict({'labels':list(),'sigma':dict()})
config['mags']['quads']['labels'] = ['qfa','qda','qdb1','qdb2','qfb','qf1','qf2','qf3','qf4']
config['mags']['quads']['sigma']['x']     = 40 * um * 1
config['mags']['quads']['sigma']['y']     = 40 * um * 1
config['mags']['quads']['sigma']['roll']  = 0.30 * mrad * 1
config['mags']['quads']['sigma']['excit'] = 0.05 * percent * 1

config['mags']['sexts'] = dict({'labels':list(),'sigma':dict()})
config['mags']['sexts']['labels'] = ['sfa','sda','sf1j','sd1j','sd2j','sd3j','sf2j',
                                     'sdb','sfb','sf1k','sd1k','sd2k','sd3k','sf2k']
config['mags']['sexts']['sigma']['x']     = 40 * um * 1
config['mags']['sexts']['sigma']['y']     = 40 * um * 1
config['mags']['sexts']['sigma']['roll']  = 0.30 * mrad * 1
config['mags']['sexts']['sigma']['excit'] = 0.05 * percent * 1

config['mags']['bc'] = dict({'labels':list(),'sigma':dict()})
config['mags']['bc']['labels'] = ['bc_lf','bc_hf']
config['mags']['bc']['sigma']['x']     = 40 * um * 1
config['mags']['bc']['sigma']['y']     = 40 * um * 1
config['mags']['bc']['sigma']['roll']  = 0.30 * mrad * 1
config['mags']['bc']['sigma']['excit'] = 0.05 * percent * 1
config['mags']['bc']['sigma']['k_dip'] = 0.10 * percent * 1

config['mags']['bendblocks'] = dict({'labels':list(),'sigma':dict()})
config['mags']['bendblocks']['labels'] = ['b1','b2']
config['mags']['bendblocks']['nrsegs'] = [1,1]
config['mags']['bendblocks']['sigma']['x']      = 40 * um * 1
config['mags']['bendblocks']['sigma']['y']      = 40 * um * 1
config['mags']['bendblocks']['sigma']['roll']   = 0.30 * mrad * 1
config['mags']['bendblocks']['sigma']['excit']  = 0.05 * percent * 1
config['mags']['bendblocks']['sigma']['k_dip']  = 0.10 * percent * 1

config['girder']['x']     = 80 * um * 1
config['girder']['y']     = 80 * um * 1;
config['girder']['roll']  = 0.2 * mrad * 1;

errors = lattice_errors.generate_errors(acc,config,fam_data=fam_data,nr_mach=20,cutoff=1,rndtype='gauss',seed=19880419)

In [ ]:
machine = lattice_errors.apply_erros(acc,errors,increment=1.0)

In [ ]:
r = lattice_errors.calc_respm_cod(acc,
                                  fam_data['bpm']['index'],
                                  fam_data['chs']['index'],
                                  fam_data['cvs']['index'],
                                  symmetry=10)

bba_ind = lattice_errors.get_bba_ind(acc,fam_data['bpm']['index'])

cor_conf = dict({'bpms':fam_data['bpm']['index'],
                 'hcms':fam_data['chs']['index'],
                 'vcms':fam_data['cvs']['index'],
                 'sext_ramp':[0,1], 'respm':r, 'bba':bba_ind,
                'bpm_err':dict({'sigma':(30e-6,30e-6),'cutoff':2})})
machine2 = [machine[i][:] for i in range(len(machine))]
gcodx, gcody = lattice_errors.correct_cod(machine2,cor_conf)